# MVTS Data Toolkit

## Demo

This demo is designed to give the user a quick tour over the software's funcionalities. Below is a list of all the things one could see in this demo:
 - Downloading a dataset of 2000 multivariate time series (mvts) instances.
 - Getting some basic statistics about your data.
 - Extracting a list of statistical features from the mvts instances.
 - ...

In [1]:
import os
import yaml
from data_retriever import DataRetriever  # for downloading data
import CONSTANTS as CONST

## Download the Dataset
In this demo we use an example dataset. In the following cells, it will be automatically downloaded. But in case something goes wrong, here is the direct link:
https://bitbucket.org/gsudmlab/mvtsdata_toolkit/downloads/petdataset_01.zip

Before we download it, let's take a quick look:

In [2]:
dr = DataRetriever(1)
print('URL:\t\t{}'.format(dr.dataset_url))
print('NAME:\t\t{}'.format(dr.dataset_name))
print('TYPE:\t\t{}'.format(dr.get_compression_type()))
print('SIZE:\t\t{}'.format(dr.get_total_size()))

URL:		https://bitbucket.org/gsudmlab/mvtsdata_toolkit/downloads/petdataset_01.zip
NAME:		petdataset_01.zip
TYPE:		application/zip
SIZE:		32M


Ready to download? This may take a few seconds, depending on your internet bandwidth. Wait for the progress bar.

In [3]:
where_to = 'temp/'
dr.retrieve(target_path = where_to)

Extracting: 100%|██████████| 2001/2001 [00:01<00:00, 1638.67it/s]


OK. Let's see how many files are available to us now.

In [4]:
dr.get_total_number_of_files()

2000

## Setup Configurations
For tasks such as feature extraction (by `feature_extractor`) and data analysis (by `mvts_data_analysis` and `extracted_features_analysis`) a configuration file must be provided. We provide one inside this package, but you can create your own and place it anywhere you wish. Let's take a look at ours which is located at `./configs/feature_extraction_configs.yml`:

In [5]:
path_to_config = './configs/feature_extraction_configs.yml'
with open(path_to_config, 'r') as f:
    print(f.read())

PATH_TO_MVTS: 'data/petdataset_01/'
PATH_TO_EXTRACTED_FEATURES: 'data/extracted_features/'
META_DATA_TAGS: ['id', 'lab', 'st', 'et']
MVTS_PARAMETERS:
  - 'TOTUSJH'
  - 'TOTBSQ'
  - 'TOTPOT'
  - 'TOTUSJZ'
  - 'ABSNJZH'
  - 'SAVNCPP'
  - 'USFLUX'
  - 'TOTFZ'
  - 'MEANPOT'
  - 'EPSZ'
  - 'MEANSHR'
  - 'SHRGT45'
  - 'MEANGAM'
  - 'MEANGBT'
  - 'MEANGBZ'
  - 'MEANGBH'
  - 'MEANJZH'
  - 'TOTFY'
  - 'MEANJZD'
  - 'MEANALP'
  - 'TOTFX'
  - 'EPSY'
  - 'EPSX'
  - 'R_VALUE'
STATISTICAL_FEATURES:
  - 'get_min'
  - 'get_max'
  - 'get_median'
  - 'get_mean'
  - 'get_stddev'
  - 'get_var'
  - 'get_skewness'
  - 'get_kurtosis'
  - 'get_no_local_maxima'
  - 'get_no_local_minima'
  - 'get_no_local_extrema'
  - 'get_no_zero_crossings'
  - 'get_mean_local_maxima_value'
  - 'get_mean_local_minima_value'
  - 'get_no_mean_local_maxima_upsurges'
  - 'get_no_mean_local_minima_downslides'
  - 'get_difference_of_mins'
  - 'get_difference_of_maxs'
  - 'get_difference_of_means'
  - 'get_difference_of_stds'
  - 'ge

Here is the break-down of the pieces:
 - `PATH_to_MVTS`: relative path to where the mvts data is stored in.
 - `PATH_TO_EXTRACTED`: relative path to where the extracted features is/will be stored in.
 - `META_DATA_TAGS`: a list of strings present in mvts file-names with specific meanings. See the README.md file for more details.
 - `MVTS_PARAMETERS`: an enumerated list of the parameter names (column names) in the mvts. Comment out those that are not needed using `#` symbol.
 - `STATISTICAL_FEATURES`: an enumerated list of the methods available through this package. This example has all features. Comment out those that are not needed using `#` symbol.
 
 In the following cells, you will see how this can be used.

## Analysis of Raw Data (MVTS Data Analysis)

- #### How many files? How large of a dataset?

Using `mvts_data_analysis` we can get an idea of the dataset we are going to work on. We start with creating an instance of a `MVTSDataAnalysis`.

In [6]:
from data_analysis.mvts_data_analysis import MVTSDataAnalysis
path_to_config = './configs/feature_extraction_configs.yml'
mvda = MVTSDataAnalysis(path_to_config)
mvda.print_stat_of_directory()

----------------------------------------
Directory:			/home/azim/CODES/PyWorkspace/mvtsdata_toolkit/data/petdataset_01/
Total no. of files:	2698
Total size:			102M
Total average:		38K
----------------------------------------


- #### Get a summary Stats of the data.

Let's now get some statistics from the content of the files. To speed up the demo, we analyze only 3 parameters (namely `TOTUSJH`, `TOTBSQ`, and `TOTPOT`), and only the first 50 mvts files.

In [7]:
params = ['TOTUSJH', 'TOTBSQ', 'TOTPOT']
n = 50
mvda.compute_summary(params_name=params, first_k=n)
mvda.summary

,Parameter-Name,Val-Count,Null-Count,mean,min,25th,50th,75th,max
0,TOTUSJH,2989,11,4.418819e+02,3.494185e+00,3.367524e+01,8.256577e+01,5.776202e+02,3.162777e+03
1,TOTBSQ,2989,11,5.641615e+09,1.983268e+07,2.550211e+08,7.174134e+08,7.636239e+09,3.848284e+10
2,TOTPOT,2989,11,8.022880e+22,1.205181e+20,1.873315e+21,5.280204e+21,6.054911e+22,7.108347e+23


... which says the length of the time series, across the 50 mvts files is 3000, including 11 `NA/NAN` values. In addition, `mean`, `min`, `max`, and three quantiles are calculated for each time series.

 - #### You have a LARGE dataset?
 A parallel version of this function is also provided to help process much larger datasets efficiently. Below, we use 4 processes to do the same thing.

In [ ]:
mvda.compute_summary_in_parallel(n_jobs=4, first_k=50, verbose=False,
                                     params_name=['TOTUSJH', 'TOTBSQ', 'TOTPOT'])
mvda.summary

**Note**: The results of the parallel and sequential versions of `mvts_data_analysis` are not exactly identical. This discrepency is due to the fact that in the parallel version, the program is designed to avoid loading the entire dataset into memory so that it is not confined to any particular data size. Therefore, it relies on some statistical estimators to approximate the percentiles with some acceptable errors. The error significantly decreases as the number of mvts files increases.

## Feature Extraction

- #### What statistical features are available?

Now that we have an idea about our raw data, let's extract some features from the data. A list of ~50 statistical features are implemented in `feature_collection`. Let's take a look at them.

In [ ]:
import features.feature_collection as fc
help(fc)

- #### How to extract these features from the data?

Time to extract a set of these features from the dataset we downloaded. Let's extract 3 simple statistical features, namely `min`, `max`, and `median`, from 3 parameters, such as `TOTUSJH`, `TOTBSQ`, and `TOTPOT`. Again, to speed up the process in this demo, we only process the first 50 mvts files.

In [ ]:
from features.feature_extractor import FeatureExtractor

fe = FeatureExtractor(path_to_config)
fe.do_extraction(features_name=['get_min', 'get_max', 'get_median'],
                 params_name=['TOTUSJH', 'TOTBSQ', 'TOTPOT'], first_k=50)
fe.df_all_features

... where each row corresponds to one mvts file, and the first 4 columns represent the extracted information from the file names.

 - #### You have a LARGE dataset?
 No worries. Using the parallel implementation of feature extraction, the process can be significantly sped up.

In [ ]:
fe.do_extraction_in_parallel(n_jobs=4,
                             features_name=['get_min', 'get_max', 'get_median'],
                             params_name=['TOTUSJH', 'TOTBSQ', 'TOTPOT'], first_k=50)
fe.df_all_features

## Extracted Features Analysis

- #### A quick look over the results?

The extracted features can be easily summarized using descriptive statistics such as `meam`, `std`, `min`, `max`, and first, second and third quartiles. In addition, any missing value can also be spotted.

In [ ]:
from data_analysis.extracted_features_analysis import ExtractedFeaturesAnalysis

efa = ExtractedFeaturesAnalysis(fe.df_all_features, exclude=['id'])
efa.compute_summary()
efa.summary

... which gives a summary statistics over every extracted feature. For instance, in row `0`, the changes of the minimum values of the parameter `TUOTUSJH`, across 50 mvts files, is described in terms of `mean`, `std`, etc. This also indicates that no `NA/NAN` or missing value was generated in the process.

## Data Normalization

The extracted features can then be normalized using four different methods. 

In [ ]:
from normalizing import normalizer

df_norm = normalizer.zero_one_normalize(df=fe.df_all_features, excluded_colnames=['id'])
df_norm

**Note**: The method argument `excluded_colnames` was used to keep the column `id` intact in the normalization process. Moreover, any other column with non-numeric values were preserved in the output.